In [1]:
import pandas as pd
import os

In [7]:
# Chemin du répertoire contenant les fichiers CSV
csv_directory = 'C:/Users/Christian/Desktop/PA/Dataset/preprocessing/'

# Liste des fichiers CSV dans le répertoire
csv_files = [os.path.join(csv_directory, f) for f in os.listdir(csv_directory) if f.endswith('.csv')]

In [8]:
columns_to_keep = [
    'id_mutation', 'date_mutation', 'nature_mutation', 'valeur_fonciere', 'adresse_numero',
    'adresse_suffixe', 'adresse_code_voie', 'adresse_nom_voie', 'code_postal',
    'code_commune', 'nom_commune', 'code_departement', 'type_local', 'surface_reelle_bati', 'surface_terrain',
    'longitude', 'latitude', 'nombre_lots'
]

In [9]:
# Conditions de filtrage
mutation_types = ['Vente', "Vente en l'état futur d'achèvement"]

In [10]:
df_list = []

In [11]:
for file in csv_files:
    df = pd.read_csv(file, dtype={'code_postal': str, 'code_commune': str, 'code_departement': str}, delimiter=',')
    
    df = df[columns_to_keep]

    df = df[df['nature_mutation'].isin(mutation_types)]
    
    if 'nombre_lots' in df.columns:
        df = df[(df['nombre_lots'] <= 1) | (df['nombre_lots'].isnull())]
    
    df = df[(df['surface_reelle_bati'].notnull()) & (df['surface_reelle_bati'] > 0)]
    df = df[(df['valeur_fonciere'].notnull()) & (df['valeur_fonciere'] > 0)]
    
    df_list.append(df)

C:\Users\Christian\AppData\Local\Temp\ipykernel_12792\641554745.py:2: DtypeWarning: Columns (14,16,17,18,20,22,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, dtype={'code_postal': str, 'code_commune': str, 'code_departement': str}, delimiter=',')
C:\Users\Christian\AppData\Local\Temp\ipykernel_12792\641554745.py:2: DtypeWarning: Columns (14,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, dtype={'code_postal': str, 'code_commune': str, 'code_departement': str}, delimiter=',')
C:\Users\Christian\AppData\Local\Temp\ipykernel_12792\641554745.py:2: DtypeWarning: Columns (14,16,17,18,20,22,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, dtype={'code_postal': str, 'code_commune': str, 'code_departement': str}, delimiter=',')
C:\Users\Christian\AppData\Local\Temp\ipykernel_12792\641554745.py:2: DtypeWarning

In [12]:
final_df = pd.concat(df_list, ignore_index=True)

# Sauvegarde du DataFrame final dans un nouveau fichier CSV
final_df.to_csv('C:/Users/Christian/Desktop/PA/Dataset/valeurs_foncieres.csv', index=False, sep=';')

# Filtre pour exclure les régions d'Outre Mer & les Type d'habitation qui sont des Locals

In [13]:
final_df['code_departement_numeric'] = pd.to_numeric(final_df['code_departement'], errors='coerce')

In [14]:
df_filtered = final_df[final_df['code_departement_numeric'] <= 95]

In [15]:
df_filtered = df_filtered.drop(columns=['code_departement_numeric'])

In [16]:
# Supprimer les lignes où 'type_local' est égal à 'Local industriel. commercial ou assimilé'
df_filtered = df_filtered[df_filtered['type_local'] != 'Local industriel. commercial ou assimilé']

In [17]:
df_filtered.to_csv('C:/Users/Christian/Desktop/PA/Dataset/valeurs_foncieres.csv', index=False, sep=';')

# Filtrer le DataFrame pour ne garder que les lignes avec des 'id_mutation' uniques

In [21]:
duplicate_ids = df_filtered['id_mutation'].value_counts()
duplicate_ids = duplicate_ids[duplicate_ids > 1].index

In [22]:
df_unique = df_filtered[~df_filtered['id_mutation'].isin(duplicate_ids)]

In [24]:
df_unique.to_csv('C:/Users/Christian/Desktop/PA/Dataset/valeurs_foncieres.csv', index=False, sep=';')

# Concatenation avec fichier departement-region

In [25]:
region_csv = 'C:/Users/Christian/Desktop/PA/Dataset/departements_regions/departements-region.csv'

In [26]:
region_mapping = pd.read_csv(region_csv)

In [27]:
print(region_mapping)

   num_dep                 dep_name                 region_name
0       01                      Ain        Auvergne-Rhône-Alpes
1       02                    Aisne             Hauts-de-France
2       03                   Allier        Auvergne-Rhône-Alpes
3       04  Alpes-de-Haute-Provence  Provence-Alpes-Côte d'Azur
4       05             Hautes-Alpes  Provence-Alpes-Côte d'Azur
..     ...                      ...                         ...
91      91                  Essonne               Île-de-France
92      92           Hauts-de-Seine               Île-de-France
93      93        Seine-Saint-Denis               Île-de-France
94      94             Val-de-Marne               Île-de-France
95      95               Val-d'Oise               Île-de-France

[96 rows x 3 columns]


In [28]:
df_unique = df_unique.merge(region_mapping, how='left', left_on='code_departement', right_on='num_dep')

In [30]:
df_unique.to_csv('C:/Users/Christian/Desktop/PA/Dataset/valeurs_foncieres_with_regions.csv', index=False, sep=';')